In [ ]:
from google.colab import drive
import pandas as pd

# Step 1: Mount your Google Drive
drive.mount('/content/drive')

# Step 2: Load your cleaned dataset
file_path = '/content/drive/MyDrive/cleaned_kcc_dataset_sampled.parquet'
df = pd.read_parquet(file_path)

# Step 3: Explore dataset
print("Loaded rows:", len(df))
df.head()


Mounted at /content/drive
Loaded rows: 40849


,Season,Category,Crop,QueryType,QueryText,KccAns,year,month
0,NA,Others,Others,Government Schemes,KALIA related query,--Advised to contact KALIA block nodal officer,2021,12
1,NA,Cereals,Paddy (Dhan),\tPlant Protection\t,Swarming Caterpillar in Paddy,Recommended to spray Lambda cyhalothrin 2.5EC ...,2019,8
2,NA,Cereals,Paddy (Dhan),Weather,Farmer asked query on Weather,NO rainfall in coming 5 days and cloudy weath...,2023,11
3,NA,Millets,Maize (Makka),\tPlant Protection\t,PROTECTION FROM BIRD,USE TAPE REEL ACROSS THE CROP FIELD TO CREATE ...,2015,8
4,NA,Others,Others,Weather,Farmer asked query on Weather,Weather forecast in Balangir district and Gudv...,2019,11


In [ ]:
!pip install langdetect transformers sentencepiece accelerate torch
from langdetect import detect
from transformers import pipeline


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=8ff6790da769f094f217870e53081e0092128cded40e2c47a58d9c45fe1688eb
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
!pip install -q langchain langgraph sentence-transformers faiss-cpu pandas pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 17.4 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')




model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NameError: name 'df' is not defined

In [ ]:
df['embedding'] = df['QueryText'].apply(lambda x: model.encode(x))

In [ ]:
import faiss
import numpy as np

dimension = len(df['embedding'][0])
index = faiss.IndexFlatIP(dimension)  # Inner product since embeddings are normalized

embeddings = np.vstack(df['embedding'].to_numpy())
faiss.normalize_L2(embeddings)  # Normalize for cosine similarity with inner product

index.add(embeddings)



In [ ]:
faiss.write_index(index, 'kcc_multilingual_index.faiss')

# Save required columns for lookup
df[['QueryText', 'KccAns', 'Season', 'Crop', 'Category']].to_parquet('kcc_metadata.parquet')


In [ ]:
def retrieve_with_filters(query, season=None, crop=None, category=None, top_k=3):
    filtered_df = df
    if season:
        filtered_df = filtered_df[filtered_df['Season'] == season]
    if crop:
        filtered_df = filtered_df[filtered_df['Crop'] == crop]
    if category:
        filtered_df = filtered_df[filtered_df['Category'] == category]

    if filtered_df.empty:
        filtered_df = df

    filtered_embeds = np.vstack(filtered_df['embedding'].to_numpy()).astype('float32')
    faiss.normalize_L2(filtered_embeds)

    q_emb = model.encode([query]).astype('float32')
    faiss.normalize_L2(q_emb)

    temp_index = faiss.IndexFlatIP(dimension)
    temp_index.add(filtered_embeds)

    distances, indices = temp_index.search(q_emb, min(top_k, len(filtered_df)))

    return filtered_df.iloc[indices[0]][['QueryText', 'KccAns', 'Season', 'Crop', 'Category']]


In [ ]:
from google.colab import files

files.download('kcc_multilingual_index.faiss')
files.download('kcc_metadata.parquet')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sentence_transformers import SentenceTransformer
SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2').save('MiniLM_local')


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer.save_pretrained("mt5_local")
model.save_pretrained("mt5_local")


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import shutil
import os

# Zip the folder
folder_to_zip = 'mt5_local'
zip_filename = folder_to_zip + '.zip'
shutil.make_archive(folder_to_zip, 'zip', folder_to_zip)

# Download the zip file
from google.colab import files
files.download(zip_filename)

# Optional: Remove the zip file after download
# os.remove(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil
import os

# Zip the folder
folder_to_zip = 'MiniLM_local'
zip_filename = folder_to_zip + '.zip'
shutil.make_archive(folder_to_zip, 'zip', folder_to_zip)

# Download the zip file
from google.colab import files
files.download(zip_filename)

# Optional: Remove the zip file after download
# os.remove(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>